<a href="https://colab.research.google.com/github/ManG0A2/MLP-class/blob/main/4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

그리드 서치를 이용한 최적화 과정 정리
1. 탐색한 매게변수를 지정
2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매게변수 조합 찾음


+ max_depth 매개변수를 사용한 하이퍼파라미터 튜닝을 어떻게 할 수 있을까? 게다가 결정 트리는 테스트해 볼 매개변수가 많다.


In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [5]:
#원래 5,197개 였던 훈련 세트가 4,157개로 줄고, 검증 세트는 1,040개가 되었다. 이제 sub_input, sub_target과 val_input, val_target을 사용해 모델을 만들고 평가해 보겠다.
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [7]:
#과대적합이다.
#검증 세트를 만드느라 훈련 세트가 줄었다. 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어진다. 그렇다고 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 들쭉날쭉하고 불안정할 것이다.
#이럴 때 교차 검증 cross validation을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.
#교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다. 그 다음 이 점수를 평균하여 최종 검증 점수를 얻는다.

from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01023245, 0.01749921, 0.01497412, 0.00955153, 0.02883959]), 'score_time': array([0.00153375, 0.00193787, 0.00144315, 0.00124788, 0.0016942 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
# 처음 2개의 키(fit_time, score_time)는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미한다. 각 키마다 5개의 숫자가 담겨 있다.
#cross_validate()함수는 기본적으로 5-폴드 교차 검증을 수행한다.

import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
#교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다. 이름은 test_score지만 검증 폴드의 점수이다. 혼동주의
'''
한 가지 주의할 점은 cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다.
앞서서는 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 따로 섞을 필요가 없다.
하지만 만약 교차 검증을 할 때 훈련 세트를 섞으려면 분할기 splitter를 지정해야한다.
사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해 준다.
cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedkFold를 사용한다.
즉 앞서 수행한 교차 검증은 다음 코드와 동일하다.
'''

from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(scores)
print(np.mean(scores['test_score']))

{'fit_time': array([0.0162921 , 0.01890373, 0.0253551 , 0.02223921, 0.01544619]), 'score_time': array([0.00157428, 0.00124145, 0.00154853, 0.007725  , 0.00720429]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


In [11]:
#10-폴드 교차 검증 수행 시
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

scores = cross_validate(dt, train_input, train_target, cv=splitter)

print(np.mean(scores['test_score']))

0.8574181117533719


In [12]:
#하이퍼파라메터

# 만약 최적의 A 값을 찾았다고 하더라도 B값이 바뀐다면 최적의 A 값도 바뀌기에 결국
#두 매개변수를 동시에 바꿔가면 최적의 값을 찾아야 하는 것이다.
#간단한 방법으로 사이킷런에서 제공하는 그리드 서치 Grid Search를 사용한다.

from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [13]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)

In [14]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
'''
결정 트리 클래스의 객체를 생성하자마자 바로 전달했다.
그다음 일반 모델을 훈련하는 것처럼 gs 객체에 fit()메서드를 호출한다.
이 메서드를 호출하면 그리드 서치 객체는 결정 트리 모델 min_impuruty_decrease 값을 바꿔가며 총 5번 실행한다.
GridSearchCV의 cv 매개변수 기본값은 5이다. 따라서 min_imputiry_decrease 값마다 5-폴드 교차 검증을 수행한다.
결국 5 x 5 = 25 개의 모델을 훈련한다.
많은 모델을 훈련하기 때문에 GridSearchCV 클래스의 n_jobs 매개변수에서 병렬 실행에 사용할 CPU 코어 수를 지정하는 것이 좋다. 이 매개변수의 기본값은 1이다. -1로 지정하면 시스템에 있는 모든 코어를 사용
'''

In [16]:
#교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 한다
#그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다. 이 모델은 gs 객체의 bestestimator 속성에 저장된다

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [18]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [19]:
#각 매개변수에서 수행한 교차 검증의 평균 점수는 cvresults 속성의 'mean_test_score'키에 저장
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [20]:
#그 다음 이 인덱스를 사용해 params키에 저장된 매개변수를 출력할 수 있다.
#이 값이 최상의 검증 점수를 만든 매개변수 조합이다. 앞에서 출력한 gs.best_params_와 동일한지 확인해 보자.

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [21]:
#이 과정을 정리해 보자면

#1. 먼저 탐색할 매개변수를 지정한다.
#2. 그다음 훈련세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다. 이 조합은 그리드 서치 객체에 저장된다.
#3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련한다. 이 모델도 그리드 서치 객체에 저장된다.

params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
         'max_depth': range(5, 20, 1),
         'min_samples_split': range(2, 100, 10)}

In [22]:
#arange() 함수는 첫 번째 매개변수 값에서 시작하여 두 번째 매개변수에 도달할 때까지 세 번째 매개변수를 계속 더한 배열을 만든다.
#코드에서는 0.0001 에서 시작하여 0.001이 될 때까지 0.0001을 계속 더한 배열이다. 0.001 은 포함되지 않기에 총 9개
#range() 도 비슷하다. 즉 총 확인해야하는 경우의 수는 9 * 15 * 10 = xx개이다.

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [23]:
#최적의 조합
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [24]:
#최상의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [25]:
# 앞에서 탐색할 매개변수의 간격을 0.0001 혹은 1로 설정했는데, 이렇게 간격을 둔 것에 특별한 근거가 없다. 이를 편하게 지정해줄 수 있는 방법은?
#이럴 때 사용 가능 한 것이 랜덤서치
#랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다.

from scipy.stats import uniform, randint

In [32]:
#stats 서브 패키지에 있는 uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다. 이를 '균등 분포에서 샘플링한다'라고 말한다. randint는 정숫값을 뽑고, uniform은 실숫값을 뽑는다.
#사용하는 방법은 같다. 0에서 10 사이의 범위를 갖는 randint 객체를 만들고 10개의 숫자를 샘플링

rgen = randint(0,10)
rgen.rvs(10)

array([8, 4, 5, 0, 0, 8, 2, 1, 0, 4])

In [36]:
#중복 삭제 & 갯수 세기 -> 즉 샘플링이 제대로 된 것을 확인 할 수 있음
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([108, 103,  81,  98,  99, 110,  96,  97, 107, 101]))

In [37]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.36122528, 0.56788764, 0.24746887, 0.97906158, 0.20796566,
       0.04625761, 0.34754541, 0.05426387, 0.60702596, 0.56207752])

In [38]:
#랜덤 서치에 randint과 uniform 클래스 객체를 넘겨주고 총 몇 번을 샘플링해서 최적의 매개변수를 찾으라고 명령
#탐색할 매개변수의 딕셔너리를 만들어 보자. 여기에서는 min_samples_leaf 매개변수를 탐색 대상에 추가하겠다. 이 매개변수는 리프 노드가 되기 위한 최소 샘플의 개수이다.
#어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않는다.

params = {'min_impurity_decrease': uniform(0.0001, 0.001),
         'max_depth': randint(20, 50),
         'min_samples_split': randint(2, 25),
         'min_samples_leaf': randint(1,25),
         }

In [39]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                                              n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7eca1f0e1090>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7eca1f0e3190>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7eca1f0e14b0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7eca1f04fac0>},
                   random_state=42)

In [40]:
#위 params에 정의된 매개변수 범위에서 총 100번(n_iter 매개변수)을 샘플링하여 교차 검증을 수행하고 최적의 매개변수 조합을 찾는다.
#앞서 그리드 서치보다 훨씬 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색할 수 있다.

print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [41]:
#최고의 교차검증 결과
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [42]:
#최적의 모델은 이미 전체 훈련 세트(train_input, train_target)로 훈련되어 bestestimator 속성에 저장되어있다.
#이 모델을 최종 모델로 결정하고 테스트 세트의 성능을 확인해 보자.
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
